In [ ]:

from poitousprint import get_online_csv, Toflit
from vega import VegaLite
import pandas as pds

toflit_client = Toflit()

origins = get_online_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSKCt7D4NgIN-5UIbwesdTyhhze2dVt0oovodSIu_sdTxGueFJ3EgkpYxh9_gIqXBQ1YjQFYppLxAOo/pub?output=csv')

origins_LaRochelle = set([o["origin_norm_ortho"] for o in origins if o["province"] in [
        "Aunis", "Poitou", "Angoumois", "Saintonge"]])

In [ ]:

# tous les flux de 89 direction pas la Rochelle et exports
all_flows = toflit_client.get_flows(best_guess_region_prodxpart='1', year='1789', export_import='Exports')

In [ ]:
values_per_direction = {}

for f in all_flows:
    direction = f['customs_region']
    value = f['value'] if f['value'] != '' else 0
    # value = int(value.split('.')[0])
    itype = f['export_import']
    year = f['year']
    if itype == 'Exports':
        if direction not in values_per_direction:
            values_per_direction[direction] = 0
        values_per_direction[direction] += float(value)

In [ ]:
# parmi tous les exports des directions quelle est la part qui vient de La Rochelle
def process_flow(flow):
    f = flow.copy()
    if f['origin'] != '' and f['origin'] in origins_LaRochelle:
        f['origin_poitou'] = 'De la région de La Rochelle'
    elif f['origin'] == '':
        f['origin_poitou'] = "Pas d'origine"
    else:
        f['origin_poitou'] = 'Hors de la région de La Rochelle'
    f['value'] = float(f['value']) if f['value'] != '' else 0
    f['value_part'] = f['value'] / values_per_direction[f['customs_region']]
    return f

VegaLite({
    "title": "Part des échanges de produits de La Rochelle par rapport au total des valeurs des exports par région",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "origin_poitou",
            "title": "origin_poitou"
        },
        "y": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-x"
        },
        "color": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Part des échanges de produits de La Rochelle par rapport au total des valeurs des exports par région"
        }
    }
}, pds.DataFrame([process_flow(flow) for flow in all_flows]))

In [ ]:
values_per_direction_la_rochelle = 0

for f in all_flows:
    direction = f['customs_region']
    value = f['value'] if f['value'] != '' else 0
    # value = int(value.split('.')[0])
    itype = f['export_import']
    year = f['year']
    if itype == 'Exports' and f['origin'] in origins_LaRochelle:
        values_per_direction_la_rochelle += float(value)

# parmi tous les exports des directions quelle est la part qui vient de La Rochelle
def process_flow(flow):
    f = flow.copy()
    if f['origin'] != '' and f['origin'] in origins_LaRochelle:
        f['origin_poitou'] = 'De la région de La Rochelle'
    elif f['origin'] == '':
        f['origin_poitou'] = "Pas d'origine"
    else:
        f['origin_poitou'] = 'Hors de la région de La Rochelle'
    f['value'] = float(f['value']) if f['value'] != '' else 0
    f['value_part'] = f['value'] / values_per_direction_la_rochelle
    return f

VegaLite({
    "title": "Part des valeurs des produits originaires de la Rochelle exportés par les autres directions des fermes",
    "mark": "bar",
    "encoding": {
        "y": {
            "type": "nominal", 
            "field": "customs_region",
            "title": "direction des fermes",
            "sort": "-x"
        },
        "x": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Portion des valeurs par rapport au total des flux impliquant des produits de la région"
        }
    }
}, pds.DataFrame([process_flow(flow) for flow in all_flows if flow['origin'] in origins_LaRochelle and flow['customs_region'] != 'La Rochelle']))

In [ ]:


VegaLite({
    "title": "Part des exports rapportée au total des produits de la région",
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "product_revolutionempire",
            "title": "type",
            "sort": "-size"
        },
        "y": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-size"
        },
        "color": {
            "field": "origin_poitou",
            "type": "nominal"
        },
        "size": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Part des exports rapportée au total des produits de la région"
        }
    }
}, pds.DataFrame([process_flow(flow) for flow in all_flows if process_flow(flow)['origin_poitou'] == "De la région de La Rochelle" and flow['customs_region'] != 'La Rochelle']))

In [ ]:

        
def process_flow_rel_region(flow):
    f = flow.copy()
    if f['origin'] != '' and f['origin'] in origins_LaRochelle:
        f['origin_poitou'] = 'De la région de La Rochelle'
    elif f['origin'] == '':
        f['origin_poitou'] = "Pas d'origine"
    else:
        f['origin_poitou'] = 'Hors de la région de La Rochelle'
    f['value'] = float(f['value']) if f['value'] != '' else 0
    f['value_part'] = f['value'] / values_per_direction_la_rochelle
    return f


VegaLite({
    "title": "Part des exports de produits originaires de La Rochelle dans les autres directions",
    "mark": "circle",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "product_revolutionempire",
            "title": "type de produit",
            "sort": "-size"
        },
        "y": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-size"
        },
        "color": {
            "field": "origin_poitou",
            "type": "nominal"
        },
        "size": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part",
            "title": "Part des exports par rapport à la valeur totale des produits de la région exportés par la France"
        }
    }
}, pds.DataFrame([process_flow_rel_region(flow) for flow in all_flows if process_flow_rel_region(flow)['origin_poitou'] == "De la région de La Rochelle" and flow['customs_region'] != 'La Rochelle']))

In [ ]:
exports_small_multiples = [process_flow_rel_region(flow) for flow in all_flows if process_flow_rel_region(flow)['origin_poitou'] == "De la région de La Rochelle" and flow['customs_region'] != 'La Rochelle']
exports_small_multiples = sorted(exports_small_multiples, key=lambda f : f["value_part"])
VegaLite({
    "title": "Part des exports de produits originaires de La Rochelle dans les autres directions, dimensionnés par rapport à la somme des ",
    "mark": "bar",
    "encoding": {
        "row": {
            "field": "customs_region",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": {
                "op": "sum",
                "field": "value_part",
                "order": "descending"
            }
        },
        "x": {
            "type": "nominal", 
            "field": "product_revolutionempire",
            "title": "type de produit",
            "sort": "-y"
        },
        "y": {
            "type": "quantitative",
            "aggregate": "sum",
            "field": "value_part"
        },
        "color": {
            "field": "product_revolutionempire",
            "type": "nominal",
            "sort": {
                "op": "sum",
                "field": "value_part",
                "order": "descending"
            }
        }
    }
}, pds.DataFrame(exports_small_multiples))